In [1]:
import numpy as np
import pandas as pd

In [3]:
train = pd.read_csv('input/sample-data/train_preprocessed.csv')
train_x = train.drop(['target'], axis=1)
train_y = train['target']
test_x = pd.read_csv('input/sample-data/test_preprocessed.csv')

28

In [4]:
#데이터 나누기
from sklearn.model_selection import KFold

In [5]:
kf = KFold(n_splits=4, shuffle= True, random_state=71)
tr_idx, va_idx = list(kf.split(train_x))[0]
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

In [6]:
import xgboost as xgb
from sklearn.metrics import log_loss 

In [11]:
def evaluate(features):
    dtrain=xgb.DMatrix(tr_x[features], label = tr_y)
    dvalid=xgb.DMatrix(va_x[features], label = va_y)
    params = {
        'objective':'binary:logistic', 
        'verbosity':0,
        'random_state':71
    }
    num_rounds= 10 
    early_stopping_rounds=3 
    watchlist= [(dtrain, 'train'), (dvalid, 'eval')]
    model = xgb.train(params, dtrain, num_boost_round= num_rounds, evals=watchlist, 
                      early_stopping_rounds= early_stopping_rounds, verbose_eval = 0)
    va_pred= model.predict(dvalid)
    score= log_loss(va_y, va_pred)
    
    return score

In [13]:
#Greedy foward selection 
best_score= 9999.0
selected = set([])

print('start greedy foward selection')

while True:
    if len(selected)==len(train_x.columns):
        break # 모든 특징이 선택되어 종료
    
    scores= []
    for feature in train_x.columns:
        if feature not in selected:
            #특징의 리스트에 대해서 정도를 평가하는 evaluate 함수로 수행 
            fs =list(selected)+[feature]
            score=evaluate(fs)
            scores.append((feature, score))
    #점수는 낮은 쪽이 좋다고 가정
    b_feature, b_score = sorted(scores, key=lambda tpl:tpl[1])[0]
    if b_score< best_score:
        selected.add(b_feature)
        best_score=b_score
        print(f'selected:{b_feature}')
        print(f'score:{b_score}')
    else:
        break #어떤 특징을 추가해도 점수가 오르지 않으므로 종료 

start greedy foward selection
selected:medical_info_a1
score:0.46387641017735004
selected:weight
score:0.4346644397482276
selected:height
score:0.39207130267471074
selected:medical_keyword_5
score:0.3689590572997928
selected:age
score:0.353796487686038
selected:medical_info_a2
score:0.3408690346375108
selected:medical_keyword_4
score:0.33171506076455115
selected:medical_keyword_2
score:0.32059751150161026
selected:medical_keyword_3
score:0.31389863365739584
selected:product
score:0.3085690851479769
selected:sex
score:0.30112869956195354
[('amount', 0.30131632409468295), ('medical_info_a3', 0.3061165024027228), ('medical_info_b1', 0.30117190991044046), ('medical_info_b2', 0.30356837757155297), ('medical_info_b3', 0.3087691929370165), ('medical_info_c1', 0.304236499273777), ('medical_info_c2', 0.30205987977683546), ('medical_keyword_1', 0.30562242489010094), ('medical_keyword_6', 0.30512450097352267), ('medical_keyword_7', 0.30119360735565426), ('medical_keyword_8', 0.30112869956195354),

In [14]:
len(scores)

17